# DFM Improvement for CL scenarios

Possible areas to tweak:
1. Use other layers concatenated (ex early layers or logit) prior to per-class PCA

2. Fine-tuning **

--------------------------------------------------
4. Thresholding + discarding + confidence ​+ novelty-rounds

5. if we take the scores that are best, are those usually true positives? Most confident predictions

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
from matplotlib import colors as mcolors
import torch
import random
import time
import torch.nn as nn
from sklearn.manifold import TSNE
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from torchvision import datasets, models, transforms
from sklearn.decomposition import PCA, FastICA
import itertools
import copy
import yaml
import pickle

from argparse import Namespace

sys.path.append('../src/')
import tforms
import feature_extraction.feature_extraction_utils as futils
from feature_extraction.Network_Latents_Wrapper import NetworkLatents
# import classifier as clf
import novelty_dfm_CL.novelty_detector as novel
import novelty_dfm_CL.novelty_eval as novelval 
import memory as mem
import utils
import datasets as dset

import datasets_utils as dsetutils


import novelty_dfm_CL.classifier as clf
import novelty_dfm_CL.novelty_utils as novelu


utils.seed_torch(0)

# Parameters defined in config yaml file

In [2]:
general_config_path = '/home/amandari/CodeDev/ProjIntel/src/configs/DFM_CL_improvement.yaml' # path to file 

with open(general_config_path) as fid:
        args = Namespace(**yaml.load(fid, Loader=yaml.SafeLoader))

torch.set_num_threads(args.num_threads)
pin_memory=False

# set up save
args = utils.config_saving(args)

# CL setup

In [3]:
start=time.time()
# 1) ----- Dataset 
if not hasattr(args, 'experiment_filepath'):
    args.experiment_filepath = None # have dictionary for defaults???
datasets_use = dset.call_dataset(args.dset_name, args.data_dir, args.experiment_dir, experiment_filepath=args.experiment_filepath, experiment_name=args.experiment_name, num_classes_first=args.num_classes_first,
                                                type_l_cifar=args.type_l_cifar, num_tasks_cifar=args.num_tasks_cifar, 
                                                scenario=args.scenario, keep_all_data=args.keep_all_data, scenario_classif=args.scenario_classif, 
                                                exp_type=args.exp_type, num_per_task=args.num_per_task,
                                                shuffle=args.shuffle_order)

if args.keep_all_data==True:
    train_datasets, train_datasets_new_only, test_datasets, list_tasks, dset_prep = datasets_use
else:
    train_datasets, test_datasets, list_tasks, dset_prep = datasets_use
args.dset_prep = dset_prep
if args.num_tasks>0:
    num_tasks = args.num_tasks
else:
    num_tasks = len(train_datasets)
test_loaders = [torch.utils.data.DataLoader(test_datasets[t], batch_size=args.batchsize_test,
                                                shuffle=True, num_workers=args.num_workers) for t in range(num_tasks)]
print('Data set up', time.time()-start)



# 2) ------ Network 
if len(args.fc_sizes)>0:
    fc_sizes = args.fc_sizes.split(",")
    fc_sizes = [int(x) for x in fc_sizes]
else:
    fc_sizes = []
network = clf.Resnet(dset_prep['total_classes'], resnet_arch=args.net_type, FC_layers=fc_sizes, base_freeze=True)
network = network.to(args.device)
dfm_inputs = args.dfm_layers_input.split(",")
dfm_layers_factors = str(args.dfm_layers_factors)
dfm_layers_factors = dfm_layers_factors.split(',')
dfm_inputs_factors = {}
for n in range(len(dfm_inputs)):
    dfm_inputs_factors[dfm_inputs[n]]=int(dfm_layers_factors[n]) #adaptive pooling 
network_inner = NetworkLatents(network, dfm_inputs, pool_factors=dfm_inputs_factors)
print('Network set up', time.time()-start)




# 3) ---- Novelty
args.detector_params['target_ind']=dset_prep['scenario_classif']
args.detector_params['device']=args.device
noveltyResults = novelval.save_novelty_results(num_tasks, args.experiment_name_plot, args.dir_save)
novelty_detector = novel.NoveltyDetector().create_detector(type=args.novelty_detector_name, params=args.detector_params)
print('Novelty Detector set up', time.time()-start)




# 4) ---- Memory (if applicable for recomputing DFM per task)
# Use raw images in memory
coreset = None
if args.coreset_size>0:
    if args.use_image_as_input:
        raw_sizes = {'svhn':32, 'cifar10':32, 'cifar100':32}
        input_size = 3*(raw_sizes[args.dset_name]**2)
    else:
        input_size = network.feat_size  
    if args.coreset_size>0 and args.keep_all_data==False:
        if args.coreset_size_MB>0:
            coreset_size = utils.memory_equivalence(args.coreset_size_MB, input_size, quantizer_dict=None)
        else:
            coreset_size = args.coreset_size
        coreset = mem.CoresetDynamic(coreset_size, target_ind= dset_prep['scenario_classif'], homog_ind=dset_prep['homog_ind'], device=args.device)
print('Memory set up', time.time()-start)


first_task [0, 1] 1 0
seq_tasks:  [[0, 1], [2], [3], [4], [5], [6], [7], [8], [9]]
*****Prep Data*****
prepared filelists 0.4205362796783447
None
tasks_filepaths_train ['../experiments/cifar10/inc_1class/train/nc_train_task_0.npy', '../experiments/cifar10/inc_1class/train/nc_train_task_1.npy', '../experiments/cifar10/inc_1class/train/nc_train_task_2.npy', '../experiments/cifar10/inc_1class/train/nc_train_task_3.npy', '../experiments/cifar10/inc_1class/train/nc_train_task_4.npy', '../experiments/cifar10/inc_1class/train/nc_train_task_5.npy', '../experiments/cifar10/inc_1class/train/nc_train_task_6.npy', '../experiments/cifar10/inc_1class/train/nc_train_task_7.npy', '../experiments/cifar10/inc_1class/train/nc_train_task_8.npy']
number tasks 9
prepared datasets 4.001075983047485
Data set up 4.00236439704895
load contrastive backbone
Will fetch activations from:
base.8, average pooled by -1
Network set up 9.320446491241455
Novelty Detector set up 9.32247805595398
Memory set up 9.3233041763

# CL Loop

In [6]:

start = time.time()
args.patience_lr = int(np.ceil(args.schedule_patience_perepoch*args.num_epochs))

per_task_results = {'scores_dist':{}, 'gt':{}}

for t in range(num_tasks):
    print('###############################')
    print('######### task %d ############'%(t))
    print('###############################')
    current_task = list_tasks[t]


    # ------- Set up data 
    if t>0 and (coreset is not None):
        batchsize_new =  int(args.batchsize*0.5)
        batchsize_old = args.batchsize - batchsize_new
        # --- wrap old data features (latent)
        dataset_old = dsetutils.DSET_wrapper_Replay(coreset.coreset_im, coreset.coreset_t, latents=coreset.coreset_latents, transform=args.tf_coreset)
        loader_old = torch.utils.data.DataLoader(dataset_old, batch_size=batchsize_old,
                                                shuffle=True, num_workers=args.num_workers)
    else:
        batchsize_new = args.batchsize
        batchsize_old = 0
        loader_old = None
    loader_new = torch.utils.data.DataLoader(train_datasets[t], batch_size=batchsize_new,
                                                shuffle=True, num_workers=args.num_workers)
    if args.keep_all_data==True:
        train_loaders_new_only = torch.utils.data.DataLoader(train_datasets_new_only[t], batch_size=args.batchsize_test,
                                                shuffle=True, num_workers=args.num_workers) 
    else:
        train_loaders_new_only = loader_new 



    # Evaluate Novelty Detector 
    if t>0:
        print('Get scores for novelty detector')
        params_score = {'layer':args.dfm_layers_input, 'feature_extractor':network_inner, 'base_apply_score':True, 'target_ind':dset_prep['scenario_classif']}
        if args.novelty_detector_name=='odin':
            results_novelty = novelval.evaluate_odin(t, novelty_detector, noveltyResults, params_score, train_loaders_new_only, test_loaders)
        elif args.novelty_detector_name=='dfm':
            results_novelty = novelval.evaluate_dfm(t, novelty_detector, noveltyResults, params_score, train_loaders_new_only, test_loaders)



    # call training loop with one liner 
    args, temp_train_loader = novelu.temporary_loader_novelty(args, loader_new, coreset)


    # if training classifier or doing finetuning of backbone 
    if args.train_clf:
        clf.train(t, args, novelty_detector, network_inner, loader_old, loader_new, test_loaders)


    print('Generate features for novelty evaluation and coreset')
    processed_data = futils.extract_features(network_inner, temp_train_loader, \
        target_ind=dset_prep['scenario_classif'], homog_ind=dset_prep['homog_ind'], 
        device=args.device, use_raw_images=args.use_image_as_input, raw_image_transform=args.add_tf)


    # ----append memory
    if t<num_tasks-1:
        if args.novelty_detector_name=='dfm':
            dfm_x = processed_data[0][args.dfm_layers_input]
            dfm_y = processed_data[1]
            if args.finetune_backbone=='all':
                novelty_detector, dfm_x, dfm_y = novelu.reprocess_data_novelty(args, dfm_x, dfm_y, network_inner, coreset)
            novelty_detector.fit_total(dfm_x.T, dfm_y)


        # ------ update coreset (if applicable)
        if coreset is not None:
            print('Append data to memory')
            coreset.append_memory(processed_data, current_task)
            print('coreset_im', coreset.coreset_im.shape)
        
    if t>0:
        # store results per task 
        per_task_results['scores_dist'][t] = np.concatenate((results_novelty.new_scores_dist, results_novelty.old_scores_dist), axis=1)

        per_task_results['gt'][t] = results_novelty.gt_concat


###############################
######### task 0 ############
###############################
Generate features for novelty evaluation and coreset
n_comp var 0.995
end fit 2.1776998043060303
Append data to memory
*******Appending to CORESET******
total_size 1000 room_new 1000 room_left 0 coreset now 1000
coreset_im torch.Size([1000, 2048])
###############################
######### task 1 ############
###############################
Get scores for novelty detector
scores_dist (2, 5000)
scores_dist (2, 2000)
old_scores_dist (2, 2000)
New task 1, AUROC = 0.956, AUPR = 0.981, AUPR_NORM = 0.981
new_scores [1.17800283 1.31865442 0.45209706 ... 0.44282067 0.60812485 1.04051352]
old_scores [0.09962666 0.10178394 0.11242108 ... 0.26372415 0.16067803 0.20280236]
DFM Results -  Auroc 0.956, Aupr 0.981, Aupr_norm 0.981
Average Accuracy per class old 0.9810
Generate features for novelty evaluation and coreset
n_comp var 0.995
end fit 1.1354477405548096
Append data to memory
*******Appending to CORE

# Save Dist Scores + Ground Truth 

In [ ]:

with open('%s/results_tasks.pickle'%(args.dir_save), 'wb') as handle:
    pickle.dump(per_task_results, handle, protocol=pickle.HIGHEST_PROTOCOL)


dict_keys(['scores_dist', 'gt'])